# Data Preprocessing



In [84]:
# utils
# transform two dict to feature vector
def dict_to_vec(dic1, dic2):
    kvec=[]
    vec1=[]
    for k, v in dic1.iteritems():
        kvec.append(k)
        vec1.append(v)
    vec2=[0]*len(kvec)
    for k, v in dic2.iteritems():
        if k in kvec:
            index=kvec.index(k)
            if v==None:
                v=0
            vec2[index]=v
        else:
            kvec.append(k)
            vec1.append(0)
            vec2.append(v)
    return vec1, vec2

In [82]:
import re
# validate the dict data for hero and item
def validate(data):
    id=[]
    for key in data:
        tid=data[key]
        id.append(tid)
    id.sort();
    flag=True
    for i in range(0, len(id)):
        if id[i]!=i:
            print i
            flag=False
    #print id
    return flag
# get the key-id pairs of hero or item data dict to be continuous
def continuous_format(data):
    i=0
    for key in data:
        data[key]=i
        i+=1

# construct item/hero-id based on json data
# reformat the id sequence
def dict_from_json(data, reindex):
    dict_data=dict()
    repeat=0
    for i in range(0, len(data)):
        #name=data[i]['name']
        name=re.sub('(npc_dota_hero_|item_)', '', data[i]['name'])
        #name=re.sub('[npc_|item_]', '', data[i]['name'])
        if name not in dict_data:
            if reindex:
                dict_data[name]=(i-repeat)
            else:
                dict_data[name]=int(data[i]['id'])
        else:
            repeat+=1
            
        
    return dict_data

# for a up-to-date item/hero list, we can http request to
# https://api.opendota.com/api/explorer?sql=SELECT%20*%20FROM%20items
heroes=json.load(open('heroes.json'))['rows']
items=json.load(open('items.json'))['rows']
#print heroes
heroes=dict_from_json(heroes, False)
items=dict_from_json(items, True)

#if validate(heroes):
#    print "heroes continuous id assignment success!"
if validate(items):
    print "items continuous id assignment success!"
#print items
#print heroes
#print items

items continuous id assignment success!


In [88]:
# based on the google drive data
# hero count=113
# item count=1020
import json
import os
import sys

fpath='./data/'

# init some coefficients
# the baseline freq coefficient
a=1.2
b=0.8

# init basic hero-item freq matrix

def maxValInDict(data):
    maxval=(-sys.maxint-1)
    for key in data:
        if data[key]>maxval:
            maxval=data[key]
    return maxval

hcount=maxValInDict(heroes)+1
icount=len(items)
print "hero count: " + str(hcount)
print "item count: " + str(icount)
basic_freq=[]
for i in range(0, hcount):
    basic_freq.append([0]*icount)

# load from data and computer base freq
# the first five heroes in the players list is radiant
for fname in os.listdir(fpath):
    data=json.load(open(fpath+fname))
    players=data['players']
    for i in range(0, len(players)):
        player=players[i]
        heroid=player['hero_id']
        purchase=player['purchase']
        if player['isRadiant']:
            win=player['radiant_win']
        else:
            win=(not player['radiant_win'])
        # be noticed that purchase is a dictionary
        for key in purchase:
            iid=items[key]
            try:
                if win: 
                    basic_freq[heroid][iid]+=a
                else: 
                    basic_freq[heroid][iid]+=b
            except IndexError as e:
                print "file name: " + fname
                print "item id: " + str(iid)
                print "hero id: " + str(heroid)


hero count: 121
item count: 292


In [85]:
# predict based on model
# the basic model is a matrix that contains weight 
# returns base h*k item recommendations
import random
from scipy import spatial as sp
import numpy as np
# if we do not consider time, rather we consider 
# all the necessary items in all time, then our method should also be effective
# in this case we may recommend more items like 15-20 that captures early-mid-final 
# k can be determined following:
# 1. get the avg total items throughout every match for every hero
# 2. guess
def base_rec(heroes, model, k):
    res=[]
    for h in heroes:
        hifreq=model[h]
        hifreq=hifreq.sort()
        res.append(hifreq[0:k])

# in matches won, recommend items, compute similarity with actual items
# OR
# in all games, find which side had items more similar to recommended, then compute winning percentage
# calculate the similarity between item purchase log
# per hero
# assumes both lists are composed of item name
# TODO: this may be extended by considering item purchase sequence
def hero_purchase_sim_calc(list1, list2):
    c1=Counter(list1)
    c2=Counter(list2)
    sim=cosine_similarity(c1, c2)
    return sim

# sim: similarity between item purchases
# ideal evaluation: P(Exactly same purchase log) 
# @hp: actual hero-purchase counter dict array
# @hp_rec: recommended hero-purchase counter dict array
# @opt: aggregation function, average and etc
# TODO: for diff heroes, we may have different weight when calc total similarity
def team_purchase_sim_calc(hp, hp_rec, norm=False, aggr_opt='avg'):
    sim_vec=[]
    tot_sim=0
    for (hp, hpr) in zip(hp, hp_rec):
        hp, hpr=dict_to_vec(hp, hpr)
        if norm:
            norm1=np.linalg.norm(hp)
            norm2=np.linalg.norm(hpr)
            hp=hp/norm1
            hpr=hpr/norm2
        sim=sp.distance.cosine(hp, hpr)
        sim_vec.append(sim)
    if aggr_opt=='avg':
        # the length should always be 5
        assert(len(hp)==5)
        tot_sim=sim_vec/len(hp)
    else:
        print "no such aggr function is pre defined!"
        tot_sim=-1
    return tot_sim




In [86]:
# testing cosine similarity calculation
from scipy import spatial as sp
import numpy as np
data=json.load(open('data/3803000030.json'))
dict1=data['players'][0]['purchase']
dict2=data['players'][1]['purchase']

vec1, vec2=dict_to_vec(dict1, dict2)

print "unnormalized: "
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim

print "normalized: "
norm1=np.linalg.norm(vec1)
norm2=np.linalg.norm(vec2)
vec1=vec1/norm1
vec2=vec2/norm2
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim


unnormalized: 
[2, 4, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 7, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 5, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]
0.270101541292
normalized: 
[ 0.27216553  0.54433105  0.13608276  0.13608276  0.13608276  0.27216553
  0.13608276  0.27216553  0.13608276  0.13608276  0.13608276  0.13608276
  0.13608276  0.13608276  0.54433105  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.27272727  0.63636364  0.          0.          0.09090909  0.          0.
  0.          0.          0.09090909  0.09090909  0.          0.09090909
  0.09090909  0.45454545  0.09090909  0.09090909  0.09090909  0.09090909
  0.09090909  0.27272727  0.09090909  0.09090909  0.09090909  0.09090909
  0.090909

In [87]:
# evaluate difference between purchase_log and purchase
data=json.load(open('data/3803000030.json'))
exist=False
for p in data['players']:
    kdict=dict()
    for i in p['purchase_log']:
        key=i['key']
        if key in kdict:
            kdict[key]+=1
        else:
            kdict[key]=1
    print "purchase: " + str(len(p['purchase'])) + "\tlog: " + str(len(kdict))
    print "keys not in purchase log: "
    for key in p['purchase']:
        if key not in kdict:
            print key
    print ""
    #print "keys not in purchase: "
    #for key in kdict:
    #    if key not in p['purchase']:
    #        print key
    #        exist=True
#print exist

purchase: 15	log: 15
keys not in purchase log: 

purchase: 31	log: 28
keys not in purchase log: 
dust
recipe_cyclone
recipe_hand_of_midas

purchase: 23	log: 21
keys not in purchase log: 
dust
recipe_yasha

purchase: 22	log: 18
keys not in purchase log: 
recipe_maelstrom
recipe_hand_of_midas
dust
recipe_diffusal_blade

purchase: 25	log: 22
keys not in purchase log: 
dust
recipe_diffusal_blade
recipe_yasha

purchase: 14	log: 12
keys not in purchase log: 
recipe_soul_ring
dust

purchase: 21	log: 21
keys not in purchase log: 

purchase: 37	log: 33
keys not in purchase log: 
dust
recipe_black_king_bar
recipe_sange
recipe_yasha

purchase: 38	log: 33
keys not in purchase log: 
recipe_manta
dust
recipe_wraith_band
recipe_yasha
recipe_force_staff

purchase: 19	log: 17
keys not in purchase log: 
recipe_force_staff
ward_dispenser

